In [1]:
import os, tarfile, pandas, json, itertools, string, array, pickle
import json, pandas, itertools
from collections import defaultdict
import classify_argument.classify_argument as cl
import nltk
from pycorenlp import StanfordCoreNLP
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, f1_score, precision_score, average_precision_score, fbeta_score, recall_score

/opt/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Stanford NLP Parser

In [2]:
s_nlp = StanfordCoreNLP('http://localhost:9010')

In [3]:
def gen_graph():
    """
    0 is premise
    1 is conclusion
    """
    values = {}
    index = []
    for source in sources:
        for root, dirs, files in os.walk(source):
            for file in files:
#                 print(file)
                vals_I = {}
                sentence = {}
                vals_I_L = {}
                vals_L1_L2 = {}
                vals_L2_L1 = {}
                vals_YA = set()
                vals_TA = set()
                vals_YA_L = {}
                vals_YA_I = {}
                vals_TA_from = {}
                vals_TA_to = {}
                not_vals_O = set()
                for line in open(os.path.join(root, file)):
                    for node in json.loads(line)['nodes']:
                        if node['type'] == 'I':
                            vals_I[node['nodeID']] = 0
                        elif node['type'] == 'L' and node['nodeID'] != '8837':
                            vals_L2_L1[node['nodeID']] = ''
                            sentence[node['nodeID']] = node['text']
                        elif node['type'] == 'CA':
                            not_vals_O.add(node['nodeID'])
                        elif node['type'] == 'YA':
                            vals_YA.add(node['nodeID'])
                        elif node['type'] == 'TA':
                            vals_TA.add(node['nodeID'])

                vals_O = set()
                graph = defaultdict(list)
                for line in open(os.path.join(root, file)):
                    for edge in json.loads(line)['edges']:
                        if edge['fromID'] in vals_I or edge['toID'] in vals_I:
                            graph[edge['fromID']].append(edge['toID'])
                        if edge['fromID'] in vals_I and edge['toID'] not in vals_I and edge['toID'] not in not_vals_O:
                            vals_O.add(edge['toID'])
                        if edge['fromID'] in vals_YA and edge['toID'] in vals_I:
                            vals_YA_I[edge['fromID']] = edge['toID']
                        elif edge['toID'] in vals_YA and edge['fromID'] in vals_L2_L1:
                            vals_YA_L[edge['toID']] = edge['fromID']
                        elif edge['fromID'] in vals_TA and edge['toID'] in vals_L2_L1:
                            vals_TA_from[edge['fromID']] = edge['toID']
                        elif edge['toID'] in vals_TA and edge['fromID'] in vals_L2_L1:
                            vals_TA_to[edge['toID']] = edge['fromID']

#                 print(vals_YA_I)
#                 print(vals_YA_L)
                for val_YA in vals_YA:
                    try:
                        vals_I_L[vals_YA_I[val_YA]] = vals_YA_L[val_YA]
                    except:
                        pass

#                 print(vals_I_L)
#                 print(vals_I)
                for val_TA in vals_TA:
                    vals_L2_L1[vals_TA_from[val_TA]] = vals_TA_to[val_TA]
                    vals_L1_L2[vals_TA_to[val_TA]] = vals_TA_from[val_TA]
                    
                root_node = ''
                for key in vals_L2_L1:
                    if vals_L2_L1[key] == '' and key in vals_L1_L2:
                        root_node = key
                        break

                vals_L_ind = {}
                vals_L_ind[root_node] = 1
                ind = 2
                cur = root_node
                while cur in vals_L1_L2:
                    vals_L_ind[vals_L1_L2[cur]] = ind
                    ind+=1
                    cur = vals_L1_L2[cur]

#                 print("00")
#                 print(vals_L1_L2)
#                 print(vals_L2_L1)

                for val in itertools.chain(vals_I, vals_O):
                    for vertex in graph[val]:
                        if vertex in vals_I:
                            vals_I[vertex] = 1
                
                final_values = []
                for key in vals_I_L:
                    sen = sentence[vals_I_L[key]].split(":")
#                     final_values[''.join(val for val in sen[1:])] = [vals_I[key], vals_L_ind[vals_I_L[key]]]
                    final_values.append([(''.join(val for val in sen[1:])).strip(), vals_I[key]])

                index.append(source + '/' + file)
                values[source + '/' + file] = final_values

    return values

In [4]:
sources = ['data/schemes/json']
global values
values = gen_graph()

In [5]:
values

{'data/schemes/json/nodeset1627.json': [['So, lending money at interest is shameful?',
   1],
  ['Aren’t the people you represent, nowadays called the consumer finance industry, traditionally known as usurers?',
   0],
  ['And isn’t the reason Dante confined them to the lowest circle of hell because they made a big profit at the expense of very poor people, which was widely regarded, quite rightly in my view, as shameful?',
   0],
  ['Well, then that takes away most of the ability of society to use future income to fund an unaffordable product right now',
   0]],
 'data/schemes/json/nodeset1629.json': [['Well, I think that’s slightly disingenuous, if I may say so',
   1],
  ['The interest rate varies by the amount of the charge for the credit but also the time over which you repay it',
   1],
  ['You don’t pay an interest rate', 1],
  ['So if you borrow some money from me, and you pay it back very quickly, the interest rate will be much higher than if you pay it back over a long period

In [6]:
def linking_words():
    link_words = []
    global values
    for key in values:
        for t_line in values[key]:
            line = t_line[0]
            link_words.append(line.split(",")[0])

    return link_words

In [7]:
linking_words()

['I mean one of the things that strikes me about this is that when you have incumbent governments',
 'But it’s so insulting',
 'They don’t want to stop it',
 'They don’t want to stop it',
 "All I'm saying is if we’re going to discuss morality",
 'But we haven’t yet attacked democracy',
 'Because they think that the only way they can win the hearts and minds of the demos [04006]',
 'They want voters who feel artificially rich',
 'because voters who feel artificially rich will probably vote for the incumbent',
 'It was their avoidance of being democrats and trying to win us politically',
 'I think it’s very interesting that capitalism is coming under such a lot of attack here',
 'I mean actually what you need to do is to take people seriously.',
 'Well',
 'The interest rate varies by the amount of the charge for the credit but also the time over which you repay it',
 'You don’t pay an interest rate',
 'So if you borrow some money from me',
 'So this activity is',
 'Only a desperate perso

In [8]:
def get_pos_tags(sentence):
    return nltk.pos_tag(nltk.word_tokenize(sentence))

In [9]:
def initialize():
    global tokenizer, features_loaded, classifier_maxent, present_tense, past_tense, support_words, against_words, file_preprocessed, features_c1, saved_file
    features_loaded = False
    classifier_maxent = cl.load_classifier(type_classifier='maxent')
    present_tense = ['VBP', 'VBZ','VBG']
    past_tense = ['VBD', 'VBN']
    support_words = ['moreover', 'furthermore', 'again', 'further', 'what is more', 'in addition', 'besides', 
                    'above all', 'as well', 'as well as', 'similarly', 'in the same way', 'really']
    against_words = ['however', 'but', 'on the other hand']
    file_preprocessed = ''
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    saved_file = ''

In [10]:
def sentence_type(sentence):
    """
    type of sentence determined by classification done using argument or not classifier
    """
    global features_loaded, features_c1
    if not features_loaded:
        features_c1 = cl.load_features()
        features_loaded = True

    features = cl.get_features(features_c1, cl._feature_dict(features_c1, sentence, get_pos_tags(sentence), fixed_features=True))
    result = classifier_maxent.predict(features)
    yield 'senType', result

In [11]:
def sentence_length(sentence):
    """
    Sentence length is greater than 12 or not
    """
    lenT = len(nltk.word_tokenize(sentence.translate(str.maketrans('', '', string.punctuation))))
    if lenT > 12:
        yield 'senLen', 1
    else:
        yield 'senLen', 0

In [12]:
def main_verb_tense(sentence):
    """
    tense of the main verb of the sentence
    """
    present = 0
    past = 1
    noverb = 2
    global present_tense, past_tense
    output = s_nlp.annotate(sentence, properties={
      'annotators': 'pos,depparse',
      'outputFormat': 'json'
      })
    e_dep = output['sentences'][0]['enhancedDependencies']
    main_verb = ''
    for item in e_dep:
        if item['dep'] == 'ROOT':
            main_verb = item['dependentGloss']
            break

    if main_verb != '':
        token_pos = output['sentences'][0]['tokens']
        for item in token_pos:
            if item['originalText'] == main_verb:
                if item['pos'] in present_tense:
                    yield 'tense', present
                elif item['pos'] in past_tense:
                    yield 'tense', past
                else:
                    yield 'tense', noverb

                break
    else:
        yield 'tense', noverb

In [13]:
def sentence_num(element):
    yield 'lineNum', element[2]

In [46]:
def rhetorical_type(sentence, file=None, current=0, length=0):
    """
    Return 0 for an support type, 1 for an against type, 2 for other
    """
    global support_words, against_words, saved_file, saved_r_types
    if(file!=saved_file):
        print('initializing')
        saved_r_types = [-2]*length
        saved_file = file

    if saved_r_types[current] != -2:
        print('--- again')
        return 'rtype', saved_r_types[current]
        
    for s_type in support_words:
#         if sentence.startswith(s_type):
#             print('support', sentence)
#             return 'rType', 0

        if s_type in sentence:
            print('support', current)
            print(sentence)
            saved_r_types[current] = 0
            return 'rtype', 0

    for a_type in against_words:
#         if sentence.startswith(a_type):
#             print('against', sentence)
#             return 'rType', 1

        if a_type in sentence:
            print('against', current)
            print(sentence)
            saved_r_types[current] = 1
            return 'rType', 1

    saved_r_types = 2
    return 'rType', 2

In [42]:
def rhetorical_type_p_n(line_num, current, file):
    prev = -2
    nex = -2
    prev_prev = -2
    nex_nex = -2
    if(current == 0):
        prev = 4
    if(current <= 1):
        prev_prev = 3
    if(current >= len(line_num)-2):
        nex_nex = 6
    if(current == len(line_num)-1):
        nex = 5

    if(prev == -2):
        prev = rhetorical_type(line_num[current-1][0], file=file, current=current-1, length=len(line_num))[1]
    if(nex == -2):
        nex = rhetorical_type(line_num[current+1][0], file=file, current=current+1, length=len(line_num))[1]

    yield 'rh1_type', int(str(prev)+str(nex))

    if(prev_prev == -2):
        prev_prev = rhetorical_type(line_num[current-2][0], file=file, current=current-2, length=len(line_num))[1]
    if(nex_nex == -2):
        nex_nex = rhetorical_type(line_num[current+2][0], file=file, current=current+2, length=len(line_num))[1]

    yield 'rh2_type', int(str(prev_prev)+str(prev)+str(nex_nex)+str(nex_nex))

In [43]:
def calculate_part(line_num, length):
    single_size = length/7
    for i, ele in enumerate(line_num):
        bounds = ele[1]
        l_part = int(bounds[0]/single_size)
        r_part = int(bounds[1]/single_size)
        if(l_part == r_part):
            line_num[i].append(l_part + 1)
        else:
            l_dif = single_size*r_part - bounds[0]
            r_dif = bounds[1] - single_size*r_part
            if(l_dif > r_dif):
                line_num[i].append(l_part + 1)
            else:
                line_num[i].append(r_part + 1)

In [44]:
def preprocess_file(file, sentences=None):
    """
    sentences will be in the form of a list where first value will be a sentence and second value will
    be whether it is an argument or premise, second value is not required here.
    Return a list where each element is of the form: sentence, (start_pos, end_pos), position_in_conversation(7 parts)
    """
    global file_preprocessed, preprocessed_data, tokenizer

    if file_preprocessed == file:
        return preprocessed_data
    else:
        file_preprocessed = file
        data = []

        with open(file, encoding='windows-1252') as f:
            data = " ".join(line.strip() for line in f)

        lines = tokenizer.tokenize(data)
#         print(lines)
        for i, line in enumerate(lines):
            line = line.strip()
            s_line = line.split(":")
            if len(s_line) > 1 and len(s_line[0].split(" ")) <= 3:
                lines[i] = (":".join(s_line[1:])).strip()

#         print(lines)
        line_num = []

        tot_length = 0
        if sentences != None:
            for i, sentence in enumerate(sentences):
                line_num.append([sentence[0].lower(), (0, 0)])
                tot_length += len(sentence[0])

            start_ind = 0
            end_ind = 0
            for i, line in enumerate(lines):
                line = line.lower()

                end_ind = start_ind + len(line) - 1
                for j, ele in enumerate(line_num):
                    if ele[1] == (0, 0):
                        if ele[0] in line:
                            k = 0
                            while k < len(line):
                                if line.startswith(ele[0], k):
                                    line_num[j][1] = (start_ind + k, start_ind + k + len(ele[0]) - 1)
                                    break
                                else:
                                    k += 1

                start_ind = end_ind + 1

            calculate_part(line_num, tot_length)

        else:
            start_ind = 0
            for i, line in enumerate(lines):
                end_ind = start_ind + len(line) - 1
                line_num.append([line.lower(), (start_ind, end_ind)])
                start_ind = end_ind + 1

            length = end_ind + 1
            calculate_part(line_num, length)

        preprocessed_data = line_num
        return line_num

In [18]:
initialize()
ans_1 = preprocess_file('data/schemes/txt/nodeset1631.txt')

/opt/anaconda3/lib/python3.5/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator LogisticRegression from version pre-0.18 when using version 0.18.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [19]:
ans_1

[["doesn’t it worry you at all that you're taking advantage of people when they are really in no position to negotiate?",
  (0, 115),
  1],
 ['they have absolutely very little choice except to take the terms that are being offered to them.',
  (116, 211),
  2],
 ['and that is totally untrue, because our customers all have bank accounts, 80% of them have credit cards.',
  (212, 315),
  3],
 ['what is actually happening is that people are choosing to pay down their long-term debt, their credit cards and their bank loans, and they are choosing to borrow small amounts of money for short periods of time, because they don’t have the security of future income that they did have before the credit crisis.',
  (316, 625),
  4]]

In [20]:
def _feature_dict(features_, fixed_features, file, sentences=None):
    typeF = ''
    if fixed_features == True:
        features = dict(features_)
        typeF = 'dict'
    else:
        features = features_
        typeF = 'defaultDict'

    preprocessed_data = preprocess_file(file, sentences)
#     print(preprocessed_data)
    feature_dicts = []
    line_num = preprocessed_data

    for i, ele in enumerate(line_num):

        sentence = ele[0]
        feature_dict = {}
        tokens = nltk.word_tokenize(sentence)
        if len(tokens) > 2:
    #         pos_tags = nltk.pos_tag(tokens)

            tag, val = rhetorical_type(sentence, file=file, current=i, length=len(line_num))
            if typeF == 'dict':
                if tag in features:
                    feature_num = features[tag]
                    feature_dict[feature_num] = val
            else:
                feature_num = features[tag]
                feature_dict[feature_num] = val    

            for tag, val in itertools.chain(sentence_length(sentence), sentence_type(sentence), main_verb_tense(sentence), sentence_num(ele), rhetorical_type_p_n(line_num, i, file)):
                if tag != '':
                    if typeF == 'dict':
                        if tag in features:
                            feature_num = features[tag]
                            feature_dict[feature_num] = val
                    else:
                        feature_num = features[tag]
                        feature_dict[feature_num] = val

            feature_dicts.append(feature_dict)

    return feature_dicts

In [21]:
def _make_int_array():
    return array.array(str("i"))

In [22]:
def initialize_new_features():
    features_ = defaultdict()
    features_.default_factory = features_.__len__
    return features_

In [23]:
def fit_transform(data_values, features_=initialize_new_features(), fixed_features=False):
    initialize()
    j_indices = []
    indptr = _make_int_array()
    values = []
    indptr.append(0)
    targets = []
    for file in data_values:
        file_s = file.split("/")
        file_txt = "/".join(file_s[:2]) + "/txt/" + file_s[-1].split(".")[0] + ".txt"
        print(file_txt)
        feature_dicts = _feature_dict(features_, fixed_features, file_txt, data_values[file])
        for i, feature_dict in enumerate(feature_dicts):
            if len(feature_dict) != 0:
                targets.append(data_values[file][i][1])
                j_indices.extend(feature_dict.keys())
                values.extend(feature_dict.values())
                indptr.append(len(j_indices))

    j_indices = np.asarray(j_indices, dtype=np.intc)
    indptr = np.frombuffer(indptr, dtype=np.intc)
    targets = np.asarray(targets, dtype=np.intc)
    X = csr_matrix((values, j_indices, indptr), shape = (len(indptr) - 1, len(features_)), dtype=np.int64)
    return X, targets, features_

In [24]:
def get_features(features_, feature_dicts):
    features = np.zeros((len(feature_dicts), len(features_)))
    for i, feature_dict in enumerate(feature_dicts):
        for key in feature_dict:
            features[i][key] = feature_dict[key]
    
    return features

In [47]:
X_t, targets_t, features_t = fit_transform(values)

/opt/anaconda3/lib/python3.5/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator LogisticRegression from version pre-0.18 when using version 0.18.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


data/schemes/txt/nodeset1747.txt
initializing


TypeError: 'int' object is not subscriptable

In [26]:
X_t.shape

(254, 5)

In [27]:
features_t

defaultdict(<method-wrapper '__len__' of collections.defaultdict object at 0x7efcfdc1ef78>,
            {'lineNum': 4, 'rType': 0, 'senLen': 1, 'senType': 2, 'tense': 3})

In [28]:
classifier_svn = SVC()
classifier_svn.fit(X_t, targets_t)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [29]:
classifier_svn.predict(get_features(features_t, _feature_dict(features_t, True, 'data/schemes/txt/nodeset1631.txt')))

array([0, 0, 0, 0], dtype=int32)

In [30]:
list(values.items())

[('data/schemes/json/nodeset1747.json',
  [['It was their avoidance of being democrats and trying to win us politically',
    0],
   ['They don’t want to stop it', 1],
   ['I mean one of the things that strikes me about this is that when you have incumbent governments, and they see that consumer credit boom mushrooming and mushrooming, what do they do?',
    0],
   ['They want voters who feel artificially rich', 1],
   ['Because they think that the only way they can win the hearts and minds of the demos [04006], which has not been true economically, is to buy us off with a sub-prime mortgage and a cheap sofa.',
    0],
   ['But it’s so insulting', 1],
   ["All I'm saying is if we’re going to discuss morality, we have to discuss the immorality of political leaders.",
    0],
   ['because voters who feel artificially rich will probably vote for the incumbent',
    0],
   ['I think it’s very interesting that capitalism is coming under such a lot of attack here',
    0],
   ['They don’t wa

In [31]:
def kFoldTest(data, classifierT = 'svc'):

    kFold = KFold(n = len(data), n_folds = 4)
    scores = []
    p_scores = []
    r_scores = []
    confusionMatrix = np.array([[0]*2]*2)

    list_dict = list(data.items())
    for train_indices, test_indices in kFold:
        print(train_indices)

        train_data = {}
        test_data = {}
        for ind in train_indices:
            train_data[list_dict[ind][0]] = list_dict[ind][1]

        for ind in test_indices:
            test_data[list_dict[ind][0]] = list_dict[ind][1]

        if(classifierT == 'multinomialNB'):
            classifier = MultinomialNB()
        elif(classifierT == 'maxent'):
            classifier = LogisticRegression()
        elif(classifierT == 'svc'):
            classifier = SVC()

        X, targets, features_ = fit_transform(train_data)
        train_y = targets
        classifier.fit(X, train_y)

        print(X.shape)
        
        X, targets, features_ = fit_transform(test_data, features_=features_, fixed_features=True)

        print(X.shape)

        predictions = classifier.predict(X)
        test_y = targets

        # print(test_y.shape)
        # print(predictions.shape)
        # print(confusion_matrix(test_y, predictions))
        confusionMatrix += confusion_matrix(test_y, predictions)
        score = f1_score(test_y, predictions, average='binary')
        p_score = precision_score(test_y, predictions, average='binary')
        r_score = recall_score(test_y, predictions, average='binary')
        
        p_scores.append(p_score)
        r_scores.append(r_score)        
        scores.append(score)

    print('Score:', sum(scores)/len(scores))
    print('Precision Score:', sum(p_scores)/len(p_scores))
    print('Recall Score:', sum(r_scores)/len(r_scores))
    print('Confusion matrix:')
    print(confusionMatrix)

In [59]:
kFoldTest(values)

[ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32]
data/schemes/txt/nodeset1825.txt


/opt/anaconda3/lib/python3.5/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator LogisticRegression from version pre-0.18 when using version 0.18.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


data/schemes/txt/nodeset1629.txt
data/schemes/txt/nodeset2051.txt
data/schemes/txt/nodeset2205.txt
data/schemes/txt/nodeset5975.txt
data/schemes/txt/nodeset6015.txt
data/schemes/txt/nodeset1728.txt
data/schemes/txt/nodeset6016.txt
data/schemes/txt/nodeset5976.txt
data/schemes/txt/nodeset1726.txt
data/schemes/txt/nodeset1630.txt
data/schemes/txt/nodeset1634.txt
data/schemes/txt/nodeset1741.txt
data/schemes/txt/nodeset1751.txt
data/schemes/txt/nodeset1627.txt
data/schemes/txt/nodeset1745.txt
data/schemes/txt/nodeset1714.txt
data/schemes/txt/nodeset4706.txt
data/schemes/txt/nodeset2049.txt
data/schemes/txt/nodeset1632.txt
data/schemes/txt/nodeset1631.txt
data/schemes/txt/nodeset1687.txt
data/schemes/txt/nodeset1730.txt
data/schemes/txt/nodeset1718.txt
(170, 7)
data/schemes/txt/nodeset1747.txt
data/schemes/txt/nodeset1724.txt
data/schemes/txt/nodeset5974.txt
data/schemes/txt/nodeset1711.txt
data/schemes/txt/nodeset1635.txt
data/schemes/txt/nodeset1633.txt
data/schemes/txt/nodeset1743.txt
d

In [60]:
kFoldTest(values, classifierT='maxent')

[ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32]


/opt/anaconda3/lib/python3.5/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator LogisticRegression from version pre-0.18 when using version 0.18.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


data/schemes/txt/nodeset1825.txt
data/schemes/txt/nodeset1629.txt
data/schemes/txt/nodeset2051.txt
data/schemes/txt/nodeset2205.txt
data/schemes/txt/nodeset5975.txt
data/schemes/txt/nodeset6015.txt
data/schemes/txt/nodeset1728.txt
data/schemes/txt/nodeset6016.txt
data/schemes/txt/nodeset5976.txt
data/schemes/txt/nodeset1726.txt
data/schemes/txt/nodeset1630.txt
data/schemes/txt/nodeset1634.txt
data/schemes/txt/nodeset1741.txt
data/schemes/txt/nodeset1751.txt
data/schemes/txt/nodeset1627.txt
data/schemes/txt/nodeset1745.txt
data/schemes/txt/nodeset1714.txt
data/schemes/txt/nodeset4706.txt
data/schemes/txt/nodeset2049.txt
data/schemes/txt/nodeset1632.txt
data/schemes/txt/nodeset1631.txt
data/schemes/txt/nodeset1687.txt
data/schemes/txt/nodeset1730.txt
data/schemes/txt/nodeset1718.txt
(170, 7)
data/schemes/txt/nodeset1747.txt
data/schemes/txt/nodeset1724.txt
data/schemes/txt/nodeset5974.txt
data/schemes/txt/nodeset1711.txt
data/schemes/txt/nodeset1635.txt
data/schemes/txt/nodeset1633.txt
d